In [1]:
import pandas as pd
from pandas import Timedelta
import numpy as np
import re

<div class="alert alert-block alert-warning">
<b>Disclaimer:</b> Note that the data we already pre-cleaned with the command line tool miller. Read the documentation provided to see what was done there.
</div>

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        Load the data in chunks
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [12]:
# Create an empty list to hold the chunks
chunks = []

# Load chunks of size 100.000
for chunk in pd.read_csv('taxi_main.csv', chunksize=100000):
    chunks.append(chunk)

# Concatenate all chunks 
df = pd.concat(chunks, ignore_index=True)

<div>
    <span style ="font-size: 40px; font-weight: bold; color: #8EB944">
        Formatting
    </span>
    
<hr style="color: #8EB944; height: 3px;background-color: #8EB944;border: none">
</div>

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        1. Map the Taxi Id's to numeric values
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [13]:
taxi_id_map = dict(zip(df['Taxi ID'].unique(),range(len(df['Taxi ID'].unique() ))))
df['Taxi ID'] = df['Taxi ID'].apply(lambda x: taxi_id_map[x])

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        2. Reduce the precision of the GPS data for consistency
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

We reduce the GPS coordinates to 10 digits for now, as this is the minimum precision all points have. We can reuse the function later to reduce the precision to 6 or 7 digits for the neural networks.

In [14]:
# use pythons regular expressions to find all the seperating '.' in a Point object. Then truncate both coordinates after 10 digits.

def format_point(point):
    if isinstance(point, str):  
        # Extract the numbers
        matches = re.findall(r'(-?\d+\.\d+)', point)
        
        # If there exactly two coordinates with a '.' , we can truncate the coordinates to 10 digits.
        if matches and len(matches) == 2:
            lon, lat = matches
            # Truncate to 10 decimal places
            lon = lon[:lon.index('.') + 11]
            lat = lat[:lat.index('.') + 11]
            return f"POINT ({lon} {lat})"
    return point

In [15]:
df.loc[:, 'Pickup Centroid Location'] = df['Pickup Centroid Location'].apply(format_point)
df.loc[:, 'Dropoff Centroid  Location'] = df['Dropoff Centroid  Location'].apply(format_point)

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        3. Change the datatype of the timestamps
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [16]:
# Convert columns to datetime
df['Trip Start Timestamp'] = pd.to_datetime(df['Trip Start Timestamp'])
df['Trip End Timestamp'] = pd.to_datetime(df['Trip End Timestamp'])

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        4. Change the datatype of the Census Tracts
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [17]:
# convert the census tracts and fill the NA's with zeros
df['Pickup Census Tract'] = pd.to_numeric(df['Pickup Census Tract'], errors='coerce').fillna(0).astype('Int64')
df['Dropoff Census Tract'] = pd.to_numeric(df['Dropoff Census Tract'], errors='coerce').fillna(0).astype('Int64')

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        5. Check whether Census Tracts can be used to fill missing location values
        </span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        5.1 Drop rides that started outside of the city of chicago


Before checking whether we can use the Pickup Census Tracts to fill missing location values, we should first get rid of rides that started outside the boundaries of the city of chicago, as these can be seen as spatial outliers.

In [18]:
## Download the census tracts for 2010 and put the GEOID's of the census tracts as a list
# get census tracts from: https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Census-Tracts-2010/5jrd-6zik #
valid_census = pd.read_csv("cens.csv")["GEOID10"].values

In [20]:
# filter the indexes where the ride has a pickup census tract that is close to- but outside of the city
outside_idx = df[(~ df['Pickup Census Tract'].isin(valid_census)) & (df['Pickup Census Tract']!=0)].index

# drop rides
df.drop(outside_idx, inplace = True)

In [21]:
len(outside_idx)

8585

We could use the same logic to filter out the rides that ended in census tracts that are just outside the city. There are roughly 72000 of these  rides, but we decide to keep them, because we are more interest in the demand of rides i.e. the pickup locations. Additionally, we did not drop the rides that have no dropoff location of dropoff census tract.


<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        5.2 Check utility of Pickup Census Tracts

We need to check whether there are instances where there was a truthy value for the 'Pickup Census Tract' and a missing values for the 'Pickup Centroid Location'. If this were the case, we could use the Pickup Census Tracts to map the missing Pickup Locations.

In [22]:
df[(df['Pickup Centroid Location'].isna()) & (df['Pickup Census Tract']!=0)]

,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Pickup Centroid Location,Dropoff Centroid Location


Since this filter is empty, we can drop the Pickup Census Tract Column, as there is a 1:1 relationship between the census tracts and the centroid location. Keeping the pickiup census tracts, wouldnt gain any additional information and the GPS data are needed. 

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        5.3 Check utility of Dropoff Census Tracts

Let us apply a similiar filter to see whether we can use 'Dropoff Census Tract's to fill some missing 'Dropoff Centroid Location' values.

In [23]:
# use the filter and show the first rows, if there are any
df[(df['Dropoff Centroid  Location'].isna()) & (df['Dropoff Census Tract']!=0)]

,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Pickup Centroid Location,Dropoff Centroid Location
8079,672,2019-01-01 03:00:00,2019-01-01 03:45:00,2820.0,0.00,17031320100,17031801500,64.00,POINT (-87.6209929134 41.8849871918),NaN
8598,1076,2019-01-01 03:15:00,2019-01-01 03:45:00,1800.0,1.50,17031320600,17031801500,60.75,POINT (-87.6221729369 41.8706073724),NaN
16835,1928,2019-01-01 13:45:00,2019-01-01 14:00:00,548.0,4.00,17031980000,17031806600,12.75,POINT (-87.9030396611 41.9790708201),NaN
17098,1764,2019-01-01 14:00:00,2019-01-01 14:00:00,577.0,3.70,17031980000,17031806600,12.00,POINT (-87.9030396611 41.9790708201),NaN
20908,1712,2019-01-01 17:30:00,2019-01-01 18:15:00,2014.0,17.73,17031980000,17031800700,36.00,POINT (-87.9030396611 41.9790708201),NaN
...,...,...,...,...,...,...,...,...,...,...
15219761,749,2019-12-30 23:45:00,2019-12-30 23:45:00,443.0,1.71,17031980100,17031820901,7.75,POINT (-87.7509342894 41.785998518),NaN
15219773,3115,2019-12-30 23:45:00,2019-12-30 23:45:00,540.0,1.50,17031980100,17031820901,7.75,POINT (-87.7509342894 41.785998518),NaN
15219984,968,2019-12-31 00:15:00,2019-12-31 00:30:00,420.0,1.70,17031980100,17031820901,7.50,POINT (-87.7509342894 41.785998518),NaN
15220193,517,2019-12-31 00:30:00,2019-12-31 00:30:00,360.0,1.60,17031980100,17031820901,7.25,POINT (-87.7509342894 41.785998518),NaN


We can observe that there are roughly 9000 rides that have a dropoff census tract but not a dropoff location. Mapping them is only worthwhile if the rides actually ended inside the city. Otherwise, we can simply treat them as the rides where no information of the ending location are known.
We can check the location of the census tracts on the website: https://www.chicagocityscape.com/maps/index.php#/?places_type=censustract&search_term=17031804603%20. Doing so for the most prevalent census tracts in the filter below, we find that these census tract are in the metropolitan area but not the city of chicago. Hence, we decide to drop them, as its only 9000 rides and mapping them would involve getting GPS data for these census tracts, as they cannont be mapped from other rides in the sample.

In [24]:
# filter the most common census track for the scenario
df[(df['Dropoff Centroid  Location'].isna()) & (df['Dropoff Census Tract']!=0)]['Dropoff Census Tract'].value_counts()

17031804603    3040
17031809400    1264
17031806600     496
17031820901     445
17031801603     278
               ... 
17031827400       1
17031816900       1
17031804803       1
17031818800       1
17031823400       1
Name: Dropoff Census Tract, Length: 230, dtype: Int64

In [25]:
# getting the indexes of  the rides that have no dropoff location but a dropoff census tract which is outside of chicago
dropoff_idx = df[(df['Dropoff Centroid  Location'].isna()) & (df['Dropoff Census Tract']!=0)].index

# drop rides
df.drop(dropoff_idx, inplace = True)

After dropping these rides, we can also drop the 'Dropoff Census Tract' Columns with the same reasoning we used for the 'Pickup Census Tract Columns' .

In [26]:
len(dropoff_idx)

8745

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        5.4 Drop the census tracts
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [52]:
df.drop(columns=['Pickup Census Tract', 'Dropoff Census Tract'], inplace=True)

Now we have reduced the spatial data to just two variables. This is reasonable, as the GPS data are the most accurate information and each centroid location corresponds to a single census tract and a single community area.

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        6. Split timestamps into 'Day' and  'Hour' variables
        </span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [27]:
df['Start Day'] = df['Trip Start Timestamp'].dt.date
df['End Day'] = df['Trip End Timestamp'].dt.date

df['Start Time'] = df['Trip Start Timestamp'].dt.time
df['End Time'] = df['Trip End Timestamp'].dt.time

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        7. Calculate estimate for the trip duration with the timestamps
        </span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

In [28]:
# calculate the trip duration with the time stamps, because some trip seconds data are erroneous
df['Trip Duration'] = df['Trip End Timestamp'] - df['Trip Start Timestamp']

<div>
    <span style ="font-size: 40px; font-weight: bold; color: #8EB944">
        Dropping outliers by looking at scenarios
    </span>
    
<hr style="color: #8EB944; height: 3px;background-color: #8EB944;border: none">
</div>

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        1. Drop rides that ended before they started


In [29]:
# filter the indexes of rides that ended before they started and subsequently drop them
inval_time = df[df['Trip End Timestamp'] < df['Trip Start Timestamp']].index
df.drop(inval_time, inplace = True)

In [31]:
len(inval_time)

329

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        2. Drop  rides with inconsistent timestamps

In [32]:
# filter the indexes of rides that started and ended in the same 15 minute interval but where the ride  took longer than 15 minutes
time_errors = df[(df['Trip End Timestamp'] == df['Trip Start Timestamp']) & 
(df['Trip Seconds']>900)].index
df.drop(time_errors, inplace = True)

In [33]:
len(time_errors)

137

After handling these scenarios, the timestamps are no longer needed and can be dropped.

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
         Drop the timestamps

In [34]:
df.drop(columns=['Trip Start Timestamp', 'Trip End Timestamp'], inplace=True)

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        3. Drop rides without valid fares


In [35]:
# filter the indexes of rides with a fare of less than 3.25$ (minimum fare) dollars and subsequently drop them
inval_fare = df[df['Fare']<3.25].index
df.drop(inval_fare, inplace = True)

# filter the indexes of rides with a fare of exactly 3.25$ but more than a mile traveled (each additional mile costs 2,25$)
inval_fare2 = df[(df['Fare']==3.25) & (df['Trip Miles'] >1)].index
df.drop(inval_fare2, inplace = True)

In [36]:
print(len(inval_fare),len(inval_fare2))

17373 1721


<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        4. Drop extremely long rides


In [37]:
# use pandas timedelta for filtering rides  that lasted  more than 8 hours and drop them, as these are likely outliers.
long_rides =  df[df['Trip Duration']>Timedelta('0 days 08:00:00')].index
df.drop(long_rides, inplace = True)

# lets also  drop  the rides that were longer than 8 hours according  to the trip seconds.
longer_rides =  df[df['Trip Seconds'] >= 28800].index
df.drop(longer_rides, inplace = True)

In [38]:
print(len(long_rides),len(longer_rides))

7325 39


<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        5. Drop missing fares, trip seconds and trip miles


As there are only 1500 missing values combined for the 3 variables: 'Fare', 'Trip Seconds', 'Trip Miles', we decide to drop them.

In [39]:
missing_miles = df[df['Trip Miles'].isna()].index
df.drop(missing_miles, inplace = True)


missing_seconds = df[df['Trip Seconds'].isna()].index
df.drop(missing_seconds, inplace = True)


missing_fares = df[df['Fare'].isna()].index
df.drop(missing_fares, inplace = True)

In [40]:
print(len(missing_seconds), len(missing_miles), len(missing_fares))

74 399 974


<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        6. Drop rides where the taxi was stationary
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

We assume that in order for a ride to be valid, the taxi has to move some time. Thus, the trip seconds or trip miles shouldnt be 0 or the dropoff location is different to the pickup location. A fare of more than 0 dollars alone doesnt constitute a valid ride, because there may have been problems with the taxometer.

In [41]:
# filter rides that have the  same start and endtime, 0 trip seconds and 0 trip miles and the same pickup and dropoff location

invalid_rides = df[(df['Trip Seconds']==0) & (df['Trip Duration'] == Timedelta('0 days 00:00:00'))
& (df['Pickup Centroid Location'] == df['Dropoff Centroid  Location'])
& (df['Trip Miles'] == 0)].index
df.drop(invalid_rides, inplace = True)

In [42]:
len(invalid_rides)

189953

<span style ="font-size: 18px; font-weight: bold;color: #43556A;">
        7. Drop rides with illogical prices
</span>
<hr style="color: #8EB944; height: 1px;background-color: #43556A;border: none">

Official prices for taxis in chicago in 2020 can be found here: https://www.chicago.gov/content/dam/city/depts/bacp/publicvehicleinfo/Chicabs/chicagotaxiplacard20200629.pdf.
The prices for 2019 cannot be found on the offical pages anymore, but we only use them as a loose orientation.
Since we only have the total fare and not all the components contributing to the fare, it is quite difficult to catch all outliers.
Due to this, we look for outliers by using the percentiles of the 'Fare' variable in combination with other variables.
A fare in the 99.9 percentile (92 Dollars or more) should correspond to a very far or long ride or someone had to pay 50 Dollars for throwing up in the cab.
We are not interested in the latter and thus filter out all the rides that paid more than 92 Dollars but were not among either the 99,5% longest rides in terms of mileage or trip duration. These percentiles may seeem arbitray but make sense, when considering the cost from the link provided above. The minimum fare is 3,25USD, each mile is 2,25USD and each minute is 0,34USD. The 99.5th percentile for the trip duration is 75 minutes, the 99.5th percentile for the trip duration is 25 miles. This yields an expected price of 3,25+(0,34*75)+(25*2,25)=85USD. This still leaves a margin of 7 dollars for the airport tax and other fees


In [43]:
# A price in the 99.9 percentile should correspond to a very far or long ride         
drop_fares = df[(df['Fare']>np.percentile(df['Fare'],99.9)) & 
                       ((df['Trip Miles'] < df['Trip Miles'].quantile(0.995)) &
                        (df['Trip Duration'] < df['Trip Duration'].quantile(0.995)))].index
# drop the outliers
df.drop(drop_fares, inplace = True)

In [44]:
len(drop_fares)

4285

<div>
    <span style ="font-size: 40px; font-weight: bold; color: #8EB944">
        Dropping outliers with quantiles
    </span>
    
<hr style="color: #8EB944; height: 3px;background-color: #8EB944;border: none">
</div>

We assume that the scenarios in the previous section did not catch all outliers, because there are probably 100 more scenarios. Since we cannot check each ride, we now remove rides by looking at the percentile values of the 2 KPI's 'Trip Miles' and 'Fare'. We do not remove outliers based on the variable 'Trip Seconds' or 'Trip Duration', because we already filtered out all the rides that were longer than 8 hours. We decide to drop all rides with 'Fares' or 'Trip Miles' more than 5 standard deviations away from the mean. 

In [45]:
df['Fare'].quantile([0.25,0.5,0.75,0.9,0.99,0.995,0.999,0.9995,0.9999, 0.99995, 0.99999])

0.25000      6.25000
0.50000      8.25000
0.75000     14.75000
0.90000     39.75000
0.99000     54.25000
0.99500     64.00000
0.99900     86.50000
0.99950     99.50000
0.99990    149.54940
0.99995    188.25000
0.99999    452.06946
Name: Fare, dtype: float64

<div class="alert alert-block alert-warning">
<b>Question:</b> All of the prices seem feasible.... Maybe don't do this? Maybe look at the quantiles in the table and choose one?
</div>

In [46]:
high_fares = df[df['Fare']>df['Fare'].mean() + 5 *df['Fare'].std()].index
df.drop(high_fares, inplace = True)

In [47]:
len(high_fares)

5279

In [48]:
df['Trip Miles'].quantile([0.25,0.5,0.75,0.9,0.99,0.995,0.999,0.9995,0.9999, 0.99995, 0.99999])

0.25000      0.60000
0.50000      1.30000
0.75000      3.30000
0.90000     13.79000
0.99000     20.84000
0.99500     24.60000
0.99900     33.16000
0.99950     36.68000
0.99990     43.47000
0.99995     45.50000
0.99999    206.96706
Name: Trip Miles, dtype: float64

In [49]:
high_mileage = df[df['Trip Miles']>df['Trip Miles'].mean() + 5 *df['Trip Miles'].std()].index
df.drop(high_mileage, inplace = True)

In [50]:
len(high_mileage)

15892

<div class="alert alert-block alert-warning">
<b>Question:</b> Should we drop extremly short rides like the ones in the filter below ?
</div>

In [ ]:
df[(df['Trip Miles']==0) & (df['Trip Duration']==Timedelta('0 days 00:00:00'))&
(df['Pickup Centroid Location']==df['Dropoff Centroid  Location'])]